In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [67]:
no_pages = 2

def get_data(pageNo):  
    headers = headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    r = requests.get('https://www.amazon.ca/Best-Sellers-Kindle-Store/zgbs/digital-text/ref=zg_bs_pg_'+str(pageNo)+'?_encoding=UTF8&pg='+str(pageNo), headers=headers)
    content = r.content
    soup = BeautifulSoup(content)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

The get_data function occasionally runs into issues with amazon's bot detection and can't consistently pull the data we need. When the function fails it returns a empty list. As I loop though the pages I use a while loop to run the function until it actually returns data. Preferably I would just have a consistent function but I don't and am not good enough at coding to fix it.

In [68]:
results = []
for i in range(1, no_pages + 1):
    a = get_data(i)
    while a == []:
        a = get_data(i)
    results.append(a)

Next I convert the scraped data into a data frame and save a copy as a csv into my repository for safekeeping :)

In [69]:
flatten = lambda l: [item for sublist in l for item in sublist]
bestsellers = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
bestsellers.to_csv('amazon_products.csv', index=False, encoding='utf-8')